In [1]:
import cptac
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import Delta_Correlation as dc
from statannot import add_stat_annotation
import numpy as np

In [2]:
luad = cptac.Luad()

In [3]:
df = dc.get_prot_trans_df(luad)
df.Tissue = [tissue.title() for tissue in df.Tissue]
df = df.rename(columns = {'Proteomics': 'Protein Abundance', 'Transcriptomics': 'RNA Abundance'})

cptac warning: Due to dropping the specified levels, dataframe now has 745 duplicated column headers. (C:\Users\nanel\Documents\GitHub\pancancerProteinMRNA\notebook_steps_Spearman\Delta_Correlation.py, line 26)
cptac warning: Due to dropping the specified levels, dataframe now has 745 duplicated column headers. (C:\Users\nanel\Documents\GitHub\pancancerProteinMRNA\notebook_steps_Spearman\Delta_Correlation.py, line 36)


In [4]:
genes = ['BANP', 'AURKB', 'MAP2K1']
df = df[df.Gene.isin(genes)]

In [5]:
diff_prot = pd.read_csv('data/Proteomics_differential_expression_df.csv')
diff_prot = diff_prot[diff_prot.Gene.isin(genes)]
diff_prot = diff_prot[diff_prot.Cancer == 'LUAD']
diff_trans = pd.read_csv('data/Transcriptomics_differential_expression_df.csv')
diff_trans = diff_trans[diff_trans.Gene.isin(genes)]
diff_trans = diff_trans[diff_trans.Cancer == 'LUAD']

In [6]:
a = 'Gene: BANP\n' + 'Cancer: LUAD\n'   + r'$\Delta$' + ' Correlation:' + u'\u2718' + '\n' + 'Diff. Expression: ' + u'\u2714'
b = 'Gene: AURKB\n' + 'Cancer: LUAD\n' + r'$\Delta$' + ' Correlation:' + u'\u2714' + '\n' + 'Diff. Expression:' + u'\u2714'
c = 'Gene: MAP2K1\n' + 'Cancer: LUAD\n'  + r'$\Delta$' + ' Correlation:' + u'\u2714' + '\n' + 'Diff. Expression:' + u'\u2718'
notes = [a,b,c]

In [7]:
sns.set_context(rc={"font.size":16,"axes.titlesize":16,"axes.labelsize":16, 'xtick.labelsize': 12, 'ytick.labelsize': 12}) 

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize=(15,15))
plt.subplots_adjust(hspace = 0.3)
i = 0
for gene, s in zip(genes, notes):
    gene_df = df[df.Gene == gene]
    trans_pval = diff_trans[diff_trans.Gene == gene].FDR
    trans_pval = np.array(trans_pval)
    prot_pval = diff_prot[diff_prot.Gene == gene].FDR
    prot_pval = np.array(prot_pval)
    plt.sca(ax[i][0])
    plt.gca().text(-0.3, 1, s, transform = plt.gca().transAxes, 
                 fontsize = 16, va = 'top', ha = 'right')
    sns.regplot(data = gene_df[gene_df.Tissue == 'Normal'], y = 'Protein Abundance', 
                   x = 'RNA Abundance')
    sns.regplot(data = gene_df[gene_df.Tissue == 'Tumor'], y = 'Protein Abundance', 
                   x = 'RNA Abundance')
    plt.sca(ax[i][1])
    g = sns.boxplot(data = gene_df, x = 'Tissue', y = 'Protein Abundance', order = ['Normal', 'Tumor'], fliersize = 0)
    test_results = add_stat_annotation(g, data=df, x='Tissue', y='Protein Abundance', order = ['Normal', 'Tumor'],
                                   box_pairs=[('Normal', 'Tumor')],
                                   pvalues = prot_pval, text_format='star',
                                   loc='outside', perform_stat_test=False, verbose = False)
    plt.sca(ax[i][2])
    g2 = sns.boxplot(data = gene_df, x = 'Tissue', y = 'RNA Abundance', order = ['Normal', 'Tumor'], fliersize = 0)
    test_results = add_stat_annotation(g2, data=df, x='Tissue', y='RNA Abundance', order = ['Normal', 'Tumor'],
                                   box_pairs=[('Normal', 'Tumor')],
                                   pvalues = trans_pval, text_format='star',
                                   loc='outside', perform_stat_test=False, verbose = False)
    i += 1

fig.tight_layout()
plt.savefig('Figures/Figure 2) Correlation vs Differential Expression.png')